In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/human-vs-llm-text-corpus/distribution.parquet
/kaggle/input/human-vs-llm-text-corpus/data.parquet
/kaggle/input/human-vs-llm-text-corpus/distribution.csv
/kaggle/input/human-vs-llm-text-corpus/prompts.csv
/kaggle/input/human-vs-llm-text-corpus/data.csv
/kaggle/input/human-vs-llm-text-corpus/prompts.parquet
/kaggle/input/splitted-800k-llm-ai-detected-text/data.parquet
/kaggle/input/splitted-800k-llm-ai-detected-text/split_text_simple_512.csv
/kaggle/input/splitted-800k-llm-ai-detected-text/132w_dataset.csv
/kaggle/input/splitted-800k-llm-ai-detected-text/30w_wiki_dataset.csv
/kaggle/input/splitted-800k-llm-ai-detected-text/split_text_improved_512.csv
/kaggle/input/splitted-800k-llm-ai-detected-text/data.csv
/kaggle/input/slimpajama-train-chunk1-sel/merged_df_sel150_600_1324132.csv


In [2]:
import pandas as pd

df_132w = pd.read_csv("/kaggle/input/slimpajama-train-chunk1-sel/merged_df_sel150_600_1324132.csv")
df_80w = pd.read_parquet("/kaggle/input/human-vs-llm-text-corpus/data.parquet")
df_30w = pd.read_csv("/kaggle/input/splitted-800k-llm-ai-detected-text/30w_wiki_dataset.csv")
df_80w_simple_512 = pd.read_csv("/kaggle/input/splitted-800k-llm-ai-detected-text/split_text_simple_512.csv")
df_80w_improved_512 = pd.read_csv("/kaggle/input/splitted-800k-llm-ai-detected-text/split_text_improved_512.csv")


df_80w.rename(columns={"source": "label"}, inplace=True)
df_80w = df_80w[["text", "label"]]
df_80w["label"] = df_80w["label"].apply(lambda x: 0 if x == "Human" else 1)

df_80w_simple_512.rename(columns={"source": "label"}, inplace=True)
df_80w_simple_512 = df_80w_simple_512[["text", "label"]]
df_80w_simple_512["label"] = df_80w_simple_512["label"].apply(lambda x: 0 if x == "Human" else 1)

df_80w_improved_512.rename(columns={"source": "label"}, inplace=True)
df_80w_improved_512 = df_80w_improved_512[["text", "label"]]
df_80w_improved_512["label"] = df_80w_improved_512["label"].apply(lambda x: 0 if x == "Human" else 1)

df_132w["label"] = df_132w["label"].apply(lambda x: 0 if x == 1 else 0)
df_132w = df_132w[["text", "label"]]

df_30w = df_30w[["text", "label"]]

df_30w.reset_index(drop=True, inplace=True)
df_30w.to_parquet("30w_dataset.parquet", index=False)
df_30w.head()

df_80w.reset_index(drop=True, inplace=True)
df_80w.to_parquet("80w_dataset.parquet", index=False)
df_80w.head()

df_80w_simple_512.reset_index(drop=True, inplace=True)
df_80w_simple_512.to_parquet("80w_simple_512_dataset.parquet", index=False)
df_80w_simple_512.head()

df_80w_improved_512.reset_index(drop=True, inplace=True)
df_80w_improved_512.to_parquet("80w_improved_512_dataset.parquet", index=False)
df_80w_improved_512.head()

df_212w = pd.concat([df_132w, df_80w])
df_212w.reset_index(drop=True, inplace=True)
df_212w.to_parquet("212w_dataset.parquet", index=False)
df_212w.head()

df_110w = pd.concat([df_80w, df_30w])
df_110w.reset_index(drop=True, inplace=True)
df_110w.to_parquet("110w_dataset.parquet", index=False)

df_242w = pd.concat([df_80w, df_30w, df_132w])
df_242w.reset_index(drop=True, inplace=True)
df_242w.to_parquet("242w_dataset.parquet", index=False)

df_242w = df_242w.drop_duplicates(subset=["text"])
train_label_0 = df_242w[df_242w["label"] == 0]
train_label_1 = df_242w[df_242w["label"] == 1]
num_sample = min(len(train_label_0), len(train_label_1))
train_label_0 = train_label_0.sample(num_sample)
train_label_1 = train_label_1.sample(num_sample)
df_242w_balanced = pd.concat([train_label_0, train_label_1])
df_242w_balanced = df_242w_balanced.sample(len(df_242w_balanced))
df_242w_balanced.reset_index(drop=True, inplace=True)
df_242w_balanced.to_parquet("242w_dataset_balanced_120w.parquet", index=False)

In [3]:
df_30w["label"].value_counts()

label
0    150000
1    150000
Name: count, dtype: int64

In [4]:
df_80w["label"].value_counts()

label
1    441230
0    347692
Name: count, dtype: int64

In [5]:
df_110w["label"].value_counts()

label
1    591230
0    497692
Name: count, dtype: int64

In [6]:
df_212w["label"].value_counts()

label
0    1671824
1     441230
Name: count, dtype: int64

In [7]:
df_242w["label"].value_counts()

label
0    1820790
1     591229
Name: count, dtype: int64

In [8]:
df_80w_simple_512["label"].value_counts()

label
0    768551
1    595266
Name: count, dtype: int64

In [9]:
df_80w_improved_512["label"].value_counts()

label
0    773426
1    593272
Name: count, dtype: int64

In [10]:
df_242w_balanced["label"].value_counts()

label
1    591229
0    591229
Name: count, dtype: int64